In [19]:
import sys

sys.path.append('../../..')

from config_data import api

In [20]:
api

In [22]:
import logging
logger = logging.getLogger(__name__)

In [23]:
logger

<Logger __main__ (WARNING)>

In [24]:
logger.info

<bound method Logger.info of <Logger __main__ (WARNING)>>

In [25]:
import pandas as pd

In [26]:
df = pd.read_csv('/Users/xiaoxinzhou/Documents/IFT6758_M2_CSV_data/all_data_categorical.csv')

In [27]:
df.shape

(282671, 19)

In [28]:
df.head()

,eventIdx,game_id,Game Seconds,Game Period,X-Coordinate,Y-Coordinate,Shot Distance,Shot Angle,Shot Type,Was Net Empty,Last Event Type,Last X-Coordinate,Last Y-Coordinate,Time from Last Event (seconds),Distance from Last Event,Is Rebound,Change in Shot Angle,Speed,Is Goal
0,5,2015020364,36,1,50,20,43.829214,117.149682,0,0,0,-78,-38,6,140.527577,False,0.0,23.421263,0
1,12,2015020364,108,1,77,6,13.416408,116.565051,0,0,1,69,-22,29,29.120440,False,0.0,1.004153,0
2,13,2015020364,115,1,86,-12,12.369317,14.036243,0,0,2,35,-2,7,51.971146,False,0.0,7.424449,0
3,18,2015020364,193,1,82,7,9.899495,135.000000,1,0,1,69,22,10,19.849433,False,0.0,1.984943,0
4,32,2015020364,309,1,48,-10,42.201896,76.293039,0,0,3,85,-11,23,37.013511,False,0.0,1.609283,0


In [31]:
df.iloc[: , :-1]

,eventIdx,game_id,Game Seconds,Game Period,X-Coordinate,Y-Coordinate,Shot Distance,Shot Angle,Shot Type,Was Net Empty,Last Event Type,Last X-Coordinate,Last Y-Coordinate,Time from Last Event (seconds),Distance from Last Event,Is Rebound,Change in Shot Angle,Speed
0,5,2015020364,36,1,50,20,43.829214,117.149682,0,0,0,-78,-38,6,140.527577,False,0.000000,23.421263
1,12,2015020364,108,1,77,6,13.416408,116.565051,0,0,1,69,-22,29,29.120440,False,0.000000,1.004153
2,13,2015020364,115,1,86,-12,12.369317,14.036243,0,0,2,35,-2,7,51.971146,False,0.000000,7.424449
3,18,2015020364,193,1,82,7,9.899495,135.000000,1,0,1,69,22,10,19.849433,False,0.000000,1.984943
4,32,2015020364,309,1,48,-10,42.201896,76.293039,0,0,3,85,-11,23,37.013511,False,0.000000,1.609283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282666,279,2018021002,3328,3,-30,18,61.684682,73.033852,2,0,0,-46,39,36,26.400758,False,0.000000,0.733354
282667,286,2018021002,3416,3,74,-2,15.132746,82.405357,1,0,5,-47,13,8,121.926207,True,9.603898,15.240776
282668,287,2018021002,3418,3,78,5,12.083046,114.443955,0,0,5,74,-2,2,8.062258,True,32.038598,4.031129
282669,292,2018021002,3483,3,-68,35,40.816663,30.963757,1,1,2,96,32,43,164.027437,False,0.000000,3.814592
